In [2]:
import pandas as pd
import numpy as np


### 시점 구분

+ (LC 방문전)---(LC 방문)---(대출 실행)--->
1. LC 방문 전 발생하는 데이터
    + 대출 신청 전 fico 점수, 리볼빙, dti, 모기지론 여부, 근속년수 등
    + 예상 부도율 예측 모델이 작동(부도 상태 분류 모델로 부도율 예측)
        + 모델은 1종오류와 2종오류를 최소화 하는 것을 목적함수로 튜닝
2. LC에 방문해 생기는 데이터
    + 신용등급, 대출 승인 금액, 대출 이자율, 매월 상환 금액 등
    + 적정 이자율 예측 모델이 작동(예상 부도율과 Fully Paid 케이스의 대출 이자율을 타겟변수로 모델 학습)
        + Fully Paid 케이스에 책정된 대출 이자율이 부도가 나지 않을 만큼 올바르게 책정된 이자율이라는 가정
        + 모델은 샤프비율을 최대화 하는 것을 목적함수로 튜닝
3. 대출이 실행 시점에 생기는 데이터
    + 실제 대출 금액
4. 대출 실행 이후 생기는 데이터
    + 현재까지 상환한 금액, 채권 추심 금액 등

### 1. 방문전 발생하는 데이터 feature selection(44개 + One hot encoding)


| Column 1                 | Column 2                 | Column 3                | Column 4                      |
|--------------------------|--------------------------|-------------------------|-------------------------------|
| term                     | emp_length               | annual_inc              | dti                           |
| delinq_2yrs              | fico_fico_avg            | inq_last_6mths          | open_acc                      |
| pub_rec                  | revol_bal                | revol_util              | total_acc                     |
| acc_now_delinq           | tot_cur_bal              | mths_since_rcnt_il      | total_bal_il                  |
| il_util                  | max_bal_bc               | all_util                | total_rev_hi_lim              |
| total_cu_tl              | avg_cur_bal              | bc_open_to_buy          | bc_util                       |
| chargeoff_within_12_mths | mort_acc                 | num_accts_ever_120_pd   | num_actv_rev_tl               |
| num_bc_sats              | num_bc_tl                | num_op_rev_tl           | num_rev_accts                 |
| num_rev_tl_bal_gt_0      | num_sats                 | pct_tl_nvr_dlq          | percent_bc_gt_75              |
| pub_rec_bankruptcies     | tax_liens                | tot_hi_cred_lim         | total_bal_ex_mort             |
| total_bc_limit           | total_il_high_credit_limit| home_ownership(One-hot) | verification_status(One-hot)  |



In [5]:
df = pd.read_csv("./data/for_grade_data.csv", memory_map=True)
df.head()


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified
0,6000.0,6000.0,6000.0,36.0,0.0797,187.94,0,4,2.0,45000.0,...,50054.0,15000.0,56511.0,True,False,False,False,False,False,True
1,35000.0,35000.0,35000.0,60.0,0.1447,822.95,2,11,2.0,94000.0,...,58284.0,69600.0,35000.0,False,False,False,False,True,True,False
2,23200.0,23200.0,23200.0,60.0,0.2499,680.82,4,23,10.0,110000.0,...,294461.0,20300.0,291465.0,True,False,False,False,False,False,True
3,4500.0,4500.0,4500.0,36.0,0.0993,145.06,1,6,10.0,35000.0,...,6435.0,30700.0,0.0,True,False,False,False,False,False,False
4,15000.0,15000.0,15000.0,36.0,0.1640,530.33,2,13,10.0,29500.0,...,131527.0,2400.0,128857.0,True,False,False,False,False,False,True


In [7]:
features = [
    'term', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_avg',
    'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    'acc_now_delinq', 'tot_cur_bal', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
    'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'total_cu_tl', 'avg_cur_bal',
    'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'mort_acc',
    'num_accts_ever_120_pd', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
    'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq',
    'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim',
    'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
    'home_ownership', "verification_status"
]

target = "loan_status"

df[features].head()

KeyError: "['home_ownership', 'verification_status'] not in index"